In [4]:
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak
from reportlab.lib.pagesizes import landscape, letter
from quantbullet.reporting.utils import register_fonts_from_package
from quantbullet.reporting import AdobeSourceFontStyles 

register_fonts_from_package()

In [2]:
# --- Create the PDF ---
doc = SimpleDocTemplate(
    "style_demo.pdf",
    pagesize=landscape(letter),
)

story = []

# --- First Page: Title + Subtitle ---
story.append(Spacer(1, 200))  # push down to vertical center
story.append(Paragraph("My Demo PDF Title", AdobeSourceFontStyles.SansTitle))
story.append(Paragraph("This is the subtitle under the title", AdobeSourceFontStyles.SansSubtitle))

# New page
story.append(PageBreak())

# --- Second Page: Style showcase ---
story.append(Paragraph( "This is paragraph one. This can be slightly longer so that we can test how this wraps the lines. The quick brown fox jumps over the lazy dog", AdobeSourceFontStyles.SerifNormal))

story.append(Paragraph("This is the second paragraph, in bold.", AdobeSourceFontStyles.SerifBold))

story.append(Paragraph("This is the third paragraph, in italics.", AdobeSourceFontStyles.SerifItalic))

story.append(Paragraph("This is the fourth paragraph, in code style.", AdobeSourceFontStyles.MonoCode))
# Build the PDF
doc.build(story)


## Model Report

In [23]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from quantbullet.preprocessing import FlatRampTransformer

In [24]:
feature_config = {
    'x1': FlatRampTransformer( knots=list( np.arange(4, 16, 1 ) ), include_bias=True ),
    'x2': FlatRampTransformer( knots=list( np.arange(14, 26, 1 ) ), include_bias=True ),
    'x3': OneHotEncoder( drop=None )
}

In [26]:
feature_config['x1'].knots

array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [5]:
from quantbullet.utils.debug import pickle_to_object

In [6]:
model = pickle_to_object('lprm_bcd.pkl')

In [43]:
doc = SimpleDocTemplate(
    "style_demo.pdf",
    pagesize=landscape(letter),
)

story = []

# story.append( Paragraph( f"{'Number of Features':<25}: {model.fit_metadata_.n_obs}", AdobeSourceFontStyles.MonoCode ) )
# story.append( Paragraph( f"{'Number of Obs':<25}: {model.fit_metadata_.n_features}", AdobeSourceFontStyles.MonoCode ) )
# story.append( Paragraph( f"{'Mean Y':<25}: {model.fit_metadata_.mean_y}", AdobeSourceFontStyles.MonoCode ) )

from reportlab.platypus import Table, TableStyle
from reportlab.lib import colors

story.append( Paragraph( "Model Fit Summary", AdobeSourceFontStyles.SerifItalic ) )

fit_summary = [
    ["Number of Features", f"{model.fit_metadata_.n_features:,}"],
    ["Number of Obs", f"{model.fit_metadata_.n_obs:,}"],
    ["Mean Y", f"{model.fit_metadata_.mean_y:,.4f}"],
    ["Model Scalar", f"{model.global_scalar_:,.4f}"],
    ["Best Loss", f"{model.best_loss_:,.4f}"],
    ['ftol', f"{model.fit_metadata_.ftol}" ],
    ['Use QR Decompsition', f"{model.fit_metadata_.cache_qr_decomp}" ],
]

fit_summary_table_style = TableStyle([
    ("FONTNAME", (0,0), (-1,-1), "SourceCodePro-Regular"),  # mono font
    ("FONTSIZE", (0,0), (-1,-1), 10),
    ("ALIGN", (1,0), (1,-1), "RIGHT"),  # numbers right-aligned
    ("BOTTOMPADDING", (0,0), (-1,-1), 2),
])

feature_config_data = []

for feature_group_name, transformer in feature_config.items():
    if isinstance(transformer, OneHotEncoder):
        feature_config_data.append( [ feature_group_name, "OneHotEncoder", "" ] )
    else:
        feature_config_data.append( [ feature_group_name, "FlatRampTransformer", f"knots={transformer.knots}" ] )
        

fit_summary_table = Table(fit_summary, colWidths=[200, 100])  # adjust widths
fit_summary_table.setStyle( fit_summary_table_style )
story.append(fit_summary_table)

story.append( Spacer(2, 20) )

feature_config_table_style = TableStyle([
    ("FONTNAME", (0,0), (-1,-1), "SourceCodePro-Regular"),
    ("FONTSIZE", (0,0), (-1,-1), 10),
    ("ALIGN", (0,0), (0,-1), "RIGHT"),    # feature names left
    ("ALIGN", (1,0), (1,-1), "CENTER"),  # transformer name centered
    ("ALIGN", (2,0), (2,-1), "LEFT"),    # params left
    ("VALIGN", (0,0), (-1,-1), "TOP"),
    ("BOTTOMPADDING", (0,0), (-1,-1), 4),
    ("TOPPADDING", (0,0), (-1,-1), 4),
    ("GRID", (0,0), (-1,-1), 0.25, colors.lightgrey),
])

feature_config_table = Table( feature_config_data, colWidths=[50, 200, 300] )
feature_config_table.setStyle( feature_config_table_style )
story.append(feature_config_table)



doc.build(story)